![alt text](awesome_logo.png "Awesome Retirement Portfolio Projector Tool")

# My Portfolio

In [10]:
import sqlalchemy as sql
import pandas as pd
import os
import datetime
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [11]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [12]:
# Get porfolio data
connection_string = 'sqlite:///awesome_app.db'
engine = sql.create_engine(connection_string, echo=False)

sql = """
  SELECT *
  FROM portfolio
"""

portfolio_df = pd.read_sql_query(sql, engine)
portfolio_df

,symbol,quantity,type
0,CSCO,100.0,Equity
1,TSLA,100.0,Equity
2,AGG,100.0,Fixed Income


In [25]:
# Set the tickers
tickers = portfolio_df["symbol"]
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"
# Set the number of years of data to fetch
num_years = 30

batch_size = 1000
num_days_data_needed = num_years * 365
first_date = datetime.datetime.now()
last_date = first_date - datetime.timedelta(days=num_days_data_needed)

def format_date_iso(date):
    return pd.Timestamp(date, tz="America/New_York").isoformat()

all_results_df = pd.DataFrame()
next_batch_date = first_date

while num_days_data_needed > 0:
    end_date = str(next_batch_date).split()[0]
    
    next_batch_date -= datetime.timedelta(days=batch_size)
    if next_batch_date < last_date:
        next_batch_date = last_date

    start_date = str(next_batch_date).split()[0]
    
    results_batch_df = alpaca.get_barset(
        tickers,
        timeframe,
        start = format_date_iso(start_date),
        end = format_date_iso(end_date),
        limit = batch_size
    ).df

    all_results_df = pd.concat([all_results_df, results_batch_df])
    num_days_data_needed -= batch_size
    
all_results_df.sort_index(inplace=True)

# Display data
all_results_df

AGG                                        CSCO  \
                             open      high       low   close   volume   open   
time                                                                            
2008-01-02 00:00:00-05:00  101.36  102.6100  101.0600  101.65   699222  26.99   
2008-01-03 00:00:00-05:00  101.62  101.8400  101.3700  101.72   369816  26.43   
2008-01-04 00:00:00-05:00  101.81  102.3700  101.8100  102.12   496988  26.37   
2008-01-07 00:00:00-05:00  102.21  102.3600  101.8300  102.26   690542  26.11   
2008-01-08 00:00:00-05:00  102.16  102.5200  101.9400  102.51  1532957  26.35   
...                           ...       ...       ...     ...      ...    ...   
2021-08-09 00:00:00-04:00  115.89  115.9600  115.5800  115.59  5003990  55.82   
2021-08-10 00:00:00-04:00  115.65  115.6500  115.4350  115.44  6692325  55.61   
2021-08-11 00:00:00-04:00  115.46  115.7000  115.3400  115.55  6423130  55.89   
2021-08-12 00:00:00-04:00  115.49  115.5799  115.4000  115.54  4248390  55.88   
2021-08-13 00:00:00-04:00  115.65  115.9700  115.6401  115.97  4455924  56.41   

                                                            TSLA          \
                            high    low  close    volume    open    high   
time                                                                       
2008-01-02 00:00:00-05:00  27.30  26.21  26.54  61009657     NaN     NaN   
2008-01-03 00:00:00-05:00  27.02  26.30  26.75  47194339     NaN     NaN   
2008-01-04 00:00:00-05:00  26.93  26.01  26.08  53986253     NaN     NaN   
2008-01-07 00:00:00-05:00  26.39  25.69  26.12  52928581     NaN     NaN   
2008-01-08 00:00:00-05:00  26.78  25.42  25.44  56461208     NaN     NaN   
...                          ...    ...    ...       ...     ...     ...   
2021-08-09 00:00:00-04:00  56.07  55.40  55.46   6079777  710.01  719.03   
2021-08-10 00:00:00-04:00  55.67  55.25  55.61   7684912  713.25  716.59   
2021-08-11 00:00:00-04:00  56.01  55.60  55.86   9959270  712.75  715.18   
2021-08-12 00:00:00-04:00  56.45  55.66  56.40   9512254  706.37  722.80   
2021-08-13 00:00:00-04:00  56.62  56.26  56.47   9428199  723.80  729.90   

                                                         
                              low     close      volume  
time                                                     
2008-01-02 00:00:00-05:00     NaN       NaN         NaN  
2008-01-03 00:00:00-05:00     NaN       NaN         NaN  
2008-01-04 00:00:00-05:00     NaN       NaN         NaN  
2008-01-07 00:00:00-05:00     NaN       NaN         NaN  
2008-01-08 00:00:00-05:00     NaN       NaN         NaN  
...                           ...       ...         ...  
2021-08-09 00:00:00-04:00  705.13  713.7300  13158590.0  
2021-08-10 00:00:00-04:00  701.88  709.9900  12217890.0  
2021-08-11 00:00:00-04:00  704.21  707.8400   9070550.0  
2021-08-12 00:00:00-04:00  699.40  722.1983  16743143.0  
2021-08-13 00:00:00-04:00  714.34  717.2800  15123022.0  

[3430 rows x 15 columns]